<a href="https://colab.research.google.com/github/dyrj/projeto_analise_dados_datagirls/blob/main/Machine_Learning_Ajuste_de_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introdução

Este notebook tem como objetivo explorar e demonstrar diferentes técnicas essenciais em Machine Learning para melhorar a performance e a confiabilidade dos modelos: o **Ajuste de Hiperparâmetros** e a **Validação Cruzada**.

Utilizaremos o popular dataset Breast Cancer para exemplificar como encontrar a melhor configuração para um modelo SVM (Support Vector Machine) e como avaliar o seu desempenho de maneira robusta, evitando o *overfitting*.

Ao final, teremos uma compreensão mais clara de como essas técnicas nos ajudam a construir modelos mais generalizáveis e eficazes.

# **Ajuste de Modelos**:

## Ajuste de Hiperparâmetros e Cross-validation

In [1]:
#Importações
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import (
    GridSearchCV, RandomizedSearchCV,
    KFold, StratifiedKFold, RepeatedKFold,
    LeaveOneOut, cross_val_score, train_test_split
)
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

!pip install scikit-optimize -q
from skopt import BayesSearchCV

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 5.6 MB/s eta 0:00:00


In [2]:
#Carregando dados

data = load_breast_cancer()
X, y = data.data, data.target

In [10]:
# Separar em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
len(y_train)

455

In [13]:
len(y_test)

114

In [14]:
X_train

array([[9.029e+00, 1.733e+01, 5.879e+01, ..., 1.750e-01, 4.228e-01,
        1.175e-01],
       [2.109e+01, 2.657e+01, 1.427e+02, ..., 2.903e-01, 4.098e-01,
        1.284e-01],
       [9.173e+00, 1.386e+01, 5.920e+01, ..., 5.087e-02, 3.282e-01,
        8.490e-02],
       ...,
       [1.429e+01, 1.682e+01, 9.030e+01, ..., 3.333e-02, 2.458e-01,
        6.120e-02],
       [1.398e+01, 1.962e+01, 9.112e+01, ..., 1.827e-01, 3.179e-01,
        1.055e-01],
       [1.218e+01, 2.052e+01, 7.722e+01, ..., 7.431e-02, 2.694e-01,
        6.878e-02]])

In [15]:
y_train

array([1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,

In [16]:
#Pipeline (padronização + modelo)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])

In [17]:
# Definir os parâmetros que serão testados
param_grid = {
    'svc__C': [0.1, 0.2, 0.3, 0.4, 0.5, 1, 10],
    'svc__kernel': ['linear', 'rbf']
}

## Grid Search

In [18]:
# Criar o GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, return_train_score=True)

# Treinar o modelo com validação cruzada
print("Iniciando busca em grade (Grid Search)...\n")
grid_search.fit(X_train, y_train)

# Resultados detalhados em um DataFrame
results = pd.DataFrame(grid_search.cv_results_)
results = results[['params', 'mean_test_score', 'std_test_score']]

print("Resultados por combinação de parâmetros:")
for i, row in results.iterrows():
    print(f"{i+1}. Parâmetros: {row['params']}, Acurácia média: {row['mean_test_score']:.4f} (+/- {row['std_test_score']:.4f})")

# Melhores parâmetros encontrados
print("\nMelhores parâmetros encontrados (Grid Search):")
print(grid_search.best_params_)

# Avaliação final no conjunto de teste
final_score = grid_search.score(X_test, y_test)
print(f"\nAcurácia no conjunto de teste com melhores parâmetros: {final_score:.4f}")

Iniciando busca em grade (Grid Search)...

Resultados por combinação de parâmetros:
1. Parâmetros: {'svc__C': 0.1, 'svc__kernel': 'linear'}, Acurácia média: 0.9736 (+/- 0.0112)
2. Parâmetros: {'svc__C': 0.1, 'svc__kernel': 'rbf'}, Acurácia média: 0.9429 (+/- 0.0213)
3. Parâmetros: {'svc__C': 0.2, 'svc__kernel': 'linear'}, Acurácia média: 0.9802 (+/- 0.0128)
4. Parâmetros: {'svc__C': 0.2, 'svc__kernel': 'rbf'}, Acurácia média: 0.9538 (+/- 0.0213)
5. Parâmetros: {'svc__C': 0.3, 'svc__kernel': 'linear'}, Acurácia média: 0.9714 (+/- 0.0192)
6. Parâmetros: {'svc__C': 0.3, 'svc__kernel': 'rbf'}, Acurácia média: 0.9582 (+/- 0.0189)
7. Parâmetros: {'svc__C': 0.4, 'svc__kernel': 'linear'}, Acurácia média: 0.9736 (+/- 0.0164)
8. Parâmetros: {'svc__C': 0.4, 'svc__kernel': 'rbf'}, Acurácia média: 0.9604 (+/- 0.0204)
9. Parâmetros: {'svc__C': 0.5, 'svc__kernel': 'linear'}, Acurácia média: 0.9758 (+/- 0.0082)
10. Parâmetros: {'svc__C': 0.5, 'svc__kernel': 'rbf'}, Acurácia média: 0.9692 (+/- 0.0201)


## Random Search

In [19]:
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=10,
    cv=5,
    random_state=42,
    return_train_score=True
)

# Treinamento com validação cruzada
print("Iniciando busca aleatória (Randomized Search)...\n")
random_search.fit(X_train, y_train)

# Resultados detalhados
results = pd.DataFrame(random_search.cv_results_)
results = results[['params', 'mean_test_score', 'std_test_score']]

print("Resultados por combinação testada:")
for i, row in results.iterrows():
    print(f"{i+1}. Parâmetros: {row['params']}, Acurácia média: {row['mean_test_score']:.4f} (+/- {row['std_test_score']:.4f})")

# Melhor combinação encontrada
print("\nMelhores parâmetros encontrados (Randomized Search):")
print(random_search.best_params_)

# Avaliação final no conjunto de teste
final_score = random_search.score(X_test, y_test)
print(f"Acurácia no conjunto de teste com melhores parâmetros: {final_score:.4f}")

Iniciando busca aleatória (Randomized Search)...

Resultados por combinação testada:
1. Parâmetros: {'svc__kernel': 'rbf', 'svc__C': 0.5}, Acurácia média: 0.9692 (+/- 0.0201)
2. Parâmetros: {'svc__kernel': 'rbf', 'svc__C': 1}, Acurácia média: 0.9736 (+/- 0.0164)
3. Parâmetros: {'svc__kernel': 'linear', 'svc__C': 0.1}, Acurácia média: 0.9736 (+/- 0.0112)
4. Parâmetros: {'svc__kernel': 'linear', 'svc__C': 10}, Acurácia média: 0.9670 (+/- 0.0184)
5. Parâmetros: {'svc__kernel': 'rbf', 'svc__C': 0.3}, Acurácia média: 0.9582 (+/- 0.0189)
6. Parâmetros: {'svc__kernel': 'linear', 'svc__C': 0.5}, Acurácia média: 0.9758 (+/- 0.0082)
7. Parâmetros: {'svc__kernel': 'linear', 'svc__C': 0.2}, Acurácia média: 0.9802 (+/- 0.0128)
8. Parâmetros: {'svc__kernel': 'rbf', 'svc__C': 0.1}, Acurácia média: 0.9429 (+/- 0.0213)
9. Parâmetros: {'svc__kernel': 'rbf', 'svc__C': 10}, Acurácia média: 0.9736 (+/- 0.0149)
10. Parâmetros: {'svc__kernel': 'linear', 'svc__C': 0.3}, Acurácia média: 0.9714 (+/- 0.0192)

Me

## Bayesian Optimization

In [20]:
opt = BayesSearchCV(
    estimator=pipeline,
    search_spaces=param_grid,
    n_iter=10,
    cv=5,
    random_state=42,
    return_train_score=True
)

# Inicia a busca bayesiana
print("Iniciando busca bayesiana (Bayesian Optimization)...\n")
opt.fit(X_train, y_train)

# Coleta dos resultados
results = pd.DataFrame(opt.cv_results_)
results = results[['params', 'mean_test_score', 'std_test_score']]

print("Resultados por iteração (Bayesian Search):")
for i, row in results.iterrows():
    print(f"{i+1}. Parâmetros: {row['params']}, Acurácia média: {row['mean_test_score']:.4f} (+/- {row['std_test_score']:.4f})")

# Melhor combinação encontrada
print("\nMelhores parâmetros encontrados (Bayesian Optimization):")
print(opt.best_params_)
print(f"Acurácia média com melhores parâmetros: {opt.best_score_:.4f}")

# Avaliação final no conjunto de teste
final_score = opt.score(X_test, y_test)
print(f"\nAcurácia no conjunto de teste com melhores parâmetros: {final_score:.4f}")

Iniciando busca bayesiana (Bayesian Optimization)...

Resultados por iteração (Bayesian Search):
1. Parâmetros: OrderedDict([('svc__C', 0.3), ('svc__kernel', 'rbf')]), Acurácia média: 0.9582 (+/- 0.0189)
2. Parâmetros: OrderedDict([('svc__C', 1), ('svc__kernel', 'rbf')]), Acurácia média: 0.9736 (+/- 0.0164)
3. Parâmetros: OrderedDict([('svc__C', 0.4), ('svc__kernel', 'rbf')]), Acurácia média: 0.9604 (+/- 0.0204)
4. Parâmetros: OrderedDict([('svc__C', 1), ('svc__kernel', 'linear')]), Acurácia média: 0.9714 (+/- 0.0149)
5. Parâmetros: OrderedDict([('svc__C', 1), ('svc__kernel', 'linear')]), Acurácia média: 0.9714 (+/- 0.0149)
6. Parâmetros: OrderedDict([('svc__C', 1), ('svc__kernel', 'rbf')]), Acurácia média: 0.9736 (+/- 0.0164)
7. Parâmetros: OrderedDict([('svc__C', 0.5), ('svc__kernel', 'rbf')]), Acurácia média: 0.9692 (+/- 0.0201)
8. Parâmetros: OrderedDict([('svc__C', 0.4), ('svc__kernel', 'rbf')]), Acurácia média: 0.9604 (+/- 0.0204)
9. Parâmetros: OrderedDict([('svc__C', 10), ('svc

# Cross-Validation

## Leave-One-Out

In [21]:
loo = LeaveOneOut()
scores_loo = []

print("Iniciando validação Leave-One-Out...\n")

for i, (train_index, test_index) in enumerate(loo.split(X), start=1):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    scores_loo.append(acc)

    print(f"Iteração {i}:")
    print(f"  Índice de teste: {test_index[0]}")
    print(f"  Valor real: {y_test[0]}, Predito: {y_pred[0]}")
    print(f"  Acurácia: {acc:.4f}\n")

# Acurácia média
print("Validação Leave-One-Out concluída.")
print(f"Acurácia média (LOO): {np.mean(scores_loo):.4f}")

Iniciando validação Leave-One-Out...

Iteração 1:
  Índice de teste: 0
  Valor real: 0, Predito: 0
  Acurácia: 1.0000

Iteração 2:
  Índice de teste: 1
  Valor real: 0, Predito: 0
  Acurácia: 1.0000

Iteração 3:
  Índice de teste: 2
  Valor real: 0, Predito: 0
  Acurácia: 1.0000

Iteração 4:
  Índice de teste: 3
  Valor real: 0, Predito: 0
  Acurácia: 1.0000

Iteração 5:
  Índice de teste: 4
  Valor real: 0, Predito: 0
  Acurácia: 1.0000

Iteração 6:
  Índice de teste: 5
  Valor real: 0, Predito: 0
  Acurácia: 1.0000

Iteração 7:
  Índice de teste: 6
  Valor real: 0, Predito: 0
  Acurácia: 1.0000

Iteração 8:
  Índice de teste: 7
  Valor real: 0, Predito: 0
  Acurácia: 1.0000

Iteração 9:
  Índice de teste: 8
  Valor real: 0, Predito: 0
  Acurácia: 1.0000

Iteração 10:
  Índice de teste: 9
  Valor real: 0, Predito: 0
  Acurácia: 1.0000

Iteração 11:
  Índice de teste: 10
  Valor real: 0, Predito: 0
  Acurácia: 1.0000

Iteração 12:
  Índice de teste: 11
  Valor real: 0, Predito: 0
  Acu

## K-Fold

In [22]:
# Definir o Stratified K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Lista para guardar os scores
scores_kf = []

print("Iniciando validação cruzada K-Fold...\n")

# Loop manual pelos folds
for fold, (train_index, test_index) in enumerate(kf.split(X, y), start=1):
    print(f"Fold {fold}")
    print(f"Índices de treino: {train_index[:10]}... ({len(train_index)} amostras)")
    print(f"Índices de teste: {test_index[:10]}... ({len(test_index)} amostras)")

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Treinar e testar o pipeline
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    scores_kf.append(acc)

    print(f"Acurácia no Fold {fold}: {acc:.4f}\n")

# Média final
print("Validação cruzada concluída.")
print(f"Acurácia média (K-Fold): {np.mean(scores_kf):.4f}")


Iniciando validação cruzada K-Fold...

Fold 1
Índices de treino: [ 0  1  3  4  5  7  8 12 13 14]... (455 amostras)
Índices de teste: [ 2  6  9 10 11 29 30 39 55 70]... (114 amostras)
Acurácia no Fold 1: 0.9825

Fold 2
Índices de treino: [ 1  2  3  4  5  6  8  9 10 11]... (455 amostras)
Índices de teste: [ 0  7 15 17 18 19 22 24 25 31]... (114 amostras)
Acurácia no Fold 2: 0.9825

Fold 3
Índices de treino: [ 0  1  2  4  6  7  8  9 10 11]... (455 amostras)
Índices de teste: [ 3  5 16 23 26 36 37 38 45 48]... (114 amostras)
Acurácia no Fold 3: 0.9737

Fold 4
Índices de treino: [ 0  1  2  3  5  6  7  8  9 10]... (455 amostras)
Índices de teste: [ 4 12 28 32 35 41 44 47 51 53]... (114 amostras)
Acurácia no Fold 4: 0.9912

Fold 5
Índices de treino: [ 0  2  3  4  5  6  7  9 10 11]... (456 amostras)
Índices de teste: [ 1  8 13 14 20 21 27 34 40 43]... (113 amostras)
Acurácia no Fold 5: 0.9735

Validação cruzada concluída.
Acurácia média (K-Fold): 0.9807


## Stratified K-Fold

In [23]:
# Definir o Stratified K-Fold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Lista para guardar os scores
scores_skf = []

print("Iniciando validação cruzada Stratified K-Fold...\n")

# Loop manual pelos folds
for fold, (train_index, test_index) in enumerate(skf.split(X, y), start=1):
    print(f"Fold {fold}")
    print(f"Índices de treino: {train_index[:10]}... ({len(train_index)} amostras)")
    print(f"Índices de teste: {test_index[:10]}... ({len(test_index)} amostras)")

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Treinar e testar o pipeline
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    scores_skf.append(acc)

    print(f"Acurácia no Fold {fold}: {acc:.4f}\n")

# Média final
print("Validação cruzada concluída.")
print(f"Acurácia média (Stratified K-Fold): {np.mean(scores_skf):.4f}")

Iniciando validação cruzada Stratified K-Fold...

Fold 1
Índices de treino: [ 0  1  2  3  4  5  6  7  9 10]... (512 amostras)
Índices de teste: [ 8 14 17 31 55 61 76 77 86 90]... (57 amostras)
Acurácia no Fold 1: 1.0000

Fold 2
Índices de treino: [ 1  2  3  4  5  6  7  8  9 10]... (512 amostras)
Índices de teste: [  0  29  65  70  75  79  84  88 104 112]... (57 amostras)
Acurácia no Fold 2: 0.9825

Fold 3
Índices de treino: [ 0  1  2  3  5  7  8  9 10 11]... (512 amostras)
Índices de teste: [ 4  6 19 21 24 68 74 80 91 93]... (57 amostras)
Acurácia no Fold 3: 0.9474

Fold 4
Índices de treino: [ 0  1  2  4  5  6  7  8  9 10]... (512 amostras)
Índices de teste: [ 3 13 23 36 38 56 57 59 63 69]... (57 amostras)
Acurácia no Fold 4: 0.9649

Fold 5
Índices de treino: [ 0  1  2  3  4  5  6  7  8 10]... (512 amostras)
Índices de teste: [ 9 22 30 37 41 43 51 62 64 81]... (57 amostras)
Acurácia no Fold 5: 0.9649

Fold 6
Índices de treino: [0 1 2 3 4 5 6 7 8 9]... (512 amostras)
Índices de teste: [

## Repeated K-Fold

In [24]:
# Definir o Repeated K-Fold
rkf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)

# Lista para guardar os scores
scores_rkf = []

print("Iniciando validação cruzada Repeated K-Fold...\n")

# Loop manual pelos folds
for fold, (train_index, test_index) in enumerate(rkf.split(X, y), start=1):
    print(f"Fold {fold}")
    print(f"Índices de treino: {train_index[:10]}... ({len(train_index)} amostras)")
    print(f"Índices de teste: {test_index[:10]}... ({len(test_index)} amostras)")

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Treinar e testar o pipeline
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    scores_rkf.append(acc)

    print(f"Acurácia no Fold {fold}: {acc:.4f}\n")

# Média final
print("Validação cruzada concluída.")
print(f"Acurácia média (Repeated K-Fold): {np.mean(scores_rkf):.4f}")


Iniciando validação cruzada Repeated K-Fold...

Fold 1
Índices de treino: [ 0  1  3  4  5  7  8 12 13 14]... (455 amostras)
Índices de teste: [ 2  6  9 10 11 29 30 39 55 70]... (114 amostras)
Acurácia no Fold 1: 0.9825

Fold 2
Índices de treino: [ 1  2  3  4  5  6  8  9 10 11]... (455 amostras)
Índices de teste: [ 0  7 15 17 18 19 22 24 25 31]... (114 amostras)
Acurácia no Fold 2: 0.9825

Fold 3
Índices de treino: [ 0  1  2  4  6  7  8  9 10 11]... (455 amostras)
Índices de teste: [ 3  5 16 23 26 36 37 38 45 48]... (114 amostras)
Acurácia no Fold 3: 0.9737

Fold 4
Índices de treino: [ 0  1  2  3  5  6  7  8  9 10]... (455 amostras)
Índices de teste: [ 4 12 28 32 35 41 44 47 51 53]... (114 amostras)
Acurácia no Fold 4: 0.9912

Fold 5
Índices de treino: [ 0  2  3  4  5  6  7  9 10 11]... (456 amostras)
Índices de teste: [ 1  8 13 14 20 21 27 34 40 43]... (113 amostras)
Acurácia no Fold 5: 0.9735

Fold 6
Índices de treino: [ 0  2  3  4  5  6  7  8  9 11]... (455 amostras)
Índices de teste